# Hello Agents 课程：智能体经典范式构建 （学习笔记）

本章节原内容来自 [Hello Agents 课程](https://datawhalechina.github.io/hello-agents/)，由 DataWhale 社区维护，感谢原作者的贡献。
https://datawhalechina.github.io/hello-agents/#/./chapter4/%E7%AC%AC%E5%9B%9B%E7%AB%A0%20%E6%99%BA%E8%83%BD%E4%BD%93%E7%BB%8F%E5%85%B8%E8%8C%83%E5%BC%8F%E6%9E%84%E5%BB%BA?id=_411-%e5%ae%89%e8%a3%85%e4%be%9d%e8%b5%96%e5%ba%93

Hello Agent教程选择最具代表性的三种智能体，并一步步从零实现它们：

- ReAct (Reasoning and Acting)： 一种将“思考”和“行动”紧密结合的范式，让智能体边想边做，动态调整。

- Plan-and-Solve： 一种“三思而后行”的范式，智能体首先生成一个完整的行动计划，然后严格执行。

- Reflection： 一种赋予智能体“反思”能力的范式，通过自我批判和修正来优化结果。

## ReAct 智能体

ReAct 智能体结合了“思考”（Reasoning）和“行动”（Acting），使其能够在执行任务的过程中动态地调整策略。以下是 ReAct 智能体的核心组件和实现步骤：

1. **环境设置**：定义智能体所处的环境，包括可用的工具和资源。
2. **思考模块**：实现一个思考模块，使智能体能够根据当前状态和目标进行推理，生成下一步的行动计划。
3. **行动模块**：实现一个行动模块，使智能体能够执行思考模块生成的计划，并根据执行结果进行反馈。
   
### 初始化

In [1]:
import openai
from openai import OpenAI
import os

In [2]:
# Set API key and base URL globally
api_key = os.getenv("OPENAI_API_KEY") # Use correct env var name from your .env or manually set it
print("OpenAI API Key:", api_key[:10] + "..." if api_key else "Not found")  # Only show first 10 chars for security

base_url = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")  # Use correct env var name from your .env
print("OpenAI Base URL:", base_url)

model = os.getenv("OPENAI_MODEL", "gpt-5-nano")  # Use correct env var name from your .env
print("OpenAI Model:", model)  # Print the model being used

# Initialize OpenAI client with API key and base URL
client = OpenAI(
    api_key=api_key,
    base_url=base_url
)


OpenAI API Key: sk-d7GDhmo...
OpenAI Base URL: https://xiaoai.plus/v1
OpenAI Model: gpt-5-nano


In [4]:
from typing import List, Dict

class HelloAgentsLLM:
    def __init__(self, model_name=model):
        self.model_name = model_name
        apiKey = os.getenv("OPENAI_API_KEY")
        baseUrl = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")
        timeout = 60
#gpt-5-nano has no temperature parameter?
    def think(self, messages: List[Dict[str, str]], temperature: float = 0) -> str:
        print("Thinking with model:", self.model_name)
        response = client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            temperature=temperature
        )
        print("Response received:", response.choices[0].message.content)
        return response.choices[0].message.content

    def generate_response(self, prompt):
        response = client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content

In [5]:
if __name__ == '__main__':
    try:
        llmClient = HelloAgentsLLM()
        
        exampleMessages = [
            {"role": "system", "content": "You are a helpful assistant that writes Python code."},
            {"role": "user", "content": "写一个快速排序算法"}
        ]
        
        print("--- 调用LLM ---")
        responseText = llmClient.think(exampleMessages)
        if responseText:
            print("\n\n--- 完整模型响应 ---")
            print(responseText)

    except ValueError as e:
        print(e)

--- 调用LLM ---
Thinking with model: gpt-5-nano
Response received: 下面给出一个简单的、原地的快速排序实现，使用 Lomuto 分区方案。时间复杂度平均 O(n log n)，最坏 O(n^2)，空间复杂度为 O(log n) 的递归栈。

代码如下（Python）：

def partition(arr, low, high):
    # 以 high 位置的元素作为 pivot
    pivot = arr[high]
    i = low
    for j in range(low, high):
        if arr[j] <= pivot:
            arr[i], arr[j] = arr[j], arr[i]
            i += 1
    arr[i], arr[high] = arr[high], arr[i]
    return i

def quicksort(arr, low=0, high=None):
    if high is None:
        high = len(arr) - 1
    if low < high:
        p = partition(arr, low, high)
        quicksort(arr, low, p - 1)
        quicksort(arr, p + 1, high)

使用示例：

lst = [3, 1, 4, 1, 5, 9, 2, 6, 5]
quicksort(lst)
print(lst)  # 输出已排序的列表

如果你想要一个非原地的版本（返回新排好序的列表），也可以提供一个基于递归划分的实现。需要的话我给你写一个。


--- 完整模型响应 ---
下面给出一个简单的、原地的快速排序实现，使用 Lomuto 分区方案。时间复杂度平均 O(n log n)，最坏 O(n^2)，空间复杂度为 O(log n) 的递归栈。

代码如下（Python）：

def partition(arr, low, high):
    # 以 high 位置的元素作为 pivot
    pivot = arr[high]
    i = low
  

### 工具定义和实现

In [6]:
!pip install google-search-results


In [7]:
from serpapi import SerpApiClient

def search(query: str) -> str:
    """
    一个基于SerpApi的实战网页搜索引擎工具。
    它会智能地解析搜索结果，优先返回直接答案或知识图谱信息。
    """
    print(f"🔍 正在执行 [SerpApi] 网页搜索: {query}")
    try:
        api_key = os.getenv("SERPAPI_API_KEY")
        if not api_key:
            return "错误:SERPAPI_API_KEY 未在 .env 文件中配置。"

        params = {
            "engine": "google",
            "q": query,
            "api_key": api_key,
            "gl": "cn",  # 国家代码
            "hl": "zh-cn", # 语言代码
        }
        
        client = SerpApiClient(params)
        results = client.get_dict()
        
        # 智能解析:优先寻找最直接的答案
        if "answer_box_list" in results:
            return "\n".join(results["answer_box_list"])
        if "answer_box" in results and "answer" in results["answer_box"]:
            return results["answer_box"]["answer"]
        if "knowledge_graph" in results and "description" in results["knowledge_graph"]:
            return results["knowledge_graph"]["description"]
        if "organic_results" in results and results["organic_results"]:
            # 如果没有直接答案，则返回前三个有机结果的摘要
            snippets = [
                f"[{i+1}] {res.get('title', '')}\n{res.get('snippet', '')}"
                for i, res in enumerate(results["organic_results"][:3])
            ]
            return "\n\n".join(snippets)
        
        return f"对不起，没有找到关于 '{query}' 的信息。"

    except Exception as e:
        return f"搜索时发生错误: {e}"


In [9]:
from typing import Dict, Any

class ToolExecutor:
    """
    一个工具执行器，负责管理和执行工具。
    """
    def __init__(self):
        self.tools: Dict[str, Dict[str, Any]] = {}

    def registerTool(self, name: str, description: str, func: callable):
        """
        向工具箱中注册一个新工具。
        """
        if name in self.tools:
            print(f"警告:工具 '{name}' 已存在，将被覆盖。")
        self.tools[name] = {"description": description, "func": func}
        print(f"工具 '{name}' 已注册。")

    def getTool(self, name: str) -> callable:
        """
        根据名称获取一个工具的执行函数。
        """
        return self.tools.get(name, {}).get("func")

    def getAvailableTools(self) -> str:
        """
        获取所有可用工具的格式化描述字符串。
        """
        return "\n".join([
            f"- {name}: {info['description']}" 
            for name, info in self.tools.items()
        ])


In [10]:
# --- 工具初始化与使用示例 ---
if __name__ == '__main__':
    # 1. 初始化工具执行器
    toolExecutor = ToolExecutor()

    # 2. 注册我们的实战搜索工具
    search_description = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
    toolExecutor.registerTool("Search", search_description, search)
    
    # 3. 打印可用的工具
    print("\n--- 可用的工具 ---")
    print(toolExecutor.getAvailableTools())

    # 4. 智能体的Action调用，这次我们问一个实时性的问题
    print("\n--- 执行 Action: Search['英伟达最新的GPU型号是什么'] ---")
    tool_name = "Search"
    tool_input = "英伟达最新的GPU型号是什么"

    tool_function = toolExecutor.getTool(tool_name)
    if tool_function:
        observation = tool_function(tool_input)
        print("--- 观察 (Observation) ---")
        print(observation)
    else:
        print(f"错误:未找到名为 '{tool_name}' 的工具。")
        


工具 'Search' 已注册。

--- 可用的工具 ---
- Search: 一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。

--- 执行 Action: Search['英伟达最新的GPU型号是什么'] ---
🔍 正在执行 [SerpApi] 网页搜索: 英伟达最新的GPU型号是什么
--- 观察 (Observation) ---
[1] 比较GeForce 系列最新一代显卡和前代显卡
比较最新一代RTX 30 系列显卡和前代的RTX 20 系列、GTX 10 和900 系列显卡。查看规格、功能、技术支持等内容。

[2] GeForce RTX 50 系列显卡
GeForce RTX™ 50 系列GPU 搭载NVIDIA Blackwell 架构，为游戏玩家和创作者带来全新玩法。RTX 50 系列具备强大的AI 算力，带来升级体验和更逼真的画面。

[3] 一文彻底读懂：英伟达GPU分类、架构演进和参数解析
Quadro系列是英伟达专业级GPU产品线，针对商业和专业应用领域进行了优化。常见的产品型号如NVIDIA RTX A6000、A5000等。 Quadro GPU具备强大的计算能力、大 ...


In [11]:
# ReAct 提示词模板
REACT_PROMPT_TEMPLATE = """
请注意，你是一个有能力调用外部工具的智能助手。

可用工具如下:
{tools}

请严格按照以下格式进行回应:

Thought: 你的思考过程，用于分析问题、拆解任务和规划下一步行动。
Action: 你决定采取的行动，必须是以下格式之一:
- `{{tool_name}}[{{tool_input}}]`:调用一个可用工具。
- `Finish[最终答案]`:当你认为已经获得最终答案时。
- 当你收集到足够的信息，能够回答用户的最终问题时，你必须在Action:字段后使用 finish(answer="...") 来输出最终答案。

现在，请开始解决以下问题:
Question: {question}
History: {history}
"""


In [14]:
class ReActAgent:
    def __init__(self, llm_client: HelloAgentsLLM, tool_executor: ToolExecutor, max_steps: int = 5):
        self.llm_client = llm_client
        self.tool_executor = tool_executor
        self.max_steps = max_steps
        self.history = []

    def run(self, question: str):
        """
        运行ReAct智能体来回答一个问题。
        """
        self.history = [] # 每次运行时重置历史记录
        current_step = 0

        while current_step < self.max_steps:
            current_step += 1
            print(f"--- 第 {current_step} 步 ---")

            # 1. 格式化提示词
            tools_desc = self.tool_executor.getAvailableTools()
            history_str = "\n".join(self.history)
            prompt = REACT_PROMPT_TEMPLATE.format(
                tools=tools_desc,
                question=question,
                history=history_str
            )

            # 2. 调用LLM进行思考
            messages = [{"role": "user", "content": prompt}]
            response_text = self.llm_client.think(messages=messages)
            
            if not response_text:
                print("错误:LLM未能返回有效响应。")
                break
            # (这段逻辑在 run 方法的 while 循环内)
            # 3. 解析LLM的输出
            thought, action = self._parse_output(response_text)
            
            if thought:
                print(f"思考: {thought}")

            if not action:
                print("警告:未能解析出有效的Action，流程终止。")
                break

            # 4. 执行Action
            if action.startswith("Finish"):
                # 如果是Finish指令，提取最终答案并结束
                final_answer = re.match(r"Finish\[(.*)\]", action).group(1)
                print(f"🎉 最终答案: {final_answer}")
                return final_answer
            
            tool_name, tool_input = self._parse_action(action)
            if not tool_name or not tool_input:
                # ... 处理无效Action格式 ...
                continue

            print(f"🎬 行动: {tool_name}[{tool_input}]")
            
            tool_function = self.tool_executor.getTool(tool_name)
            if not tool_function:
                observation = f"错误:未找到名为 '{tool_name}' 的工具。"
            else:
                observation = tool_function(tool_input) # 调用真实工具

            ## (这段逻辑紧随工具调用之后，在 while 循环的末尾)
            print(f"👀 观察: {observation}")
            
            # 将本轮的Action和Observation添加到历史记录中
            self.history.append(f"Action: {action}")
            self.history.append(f"Observation: {observation}")

        # 循环结束
        print("已达到最大步数，流程终止。")
        return None

# (这些方法是 ReActAgent 类的一部分)
    def _parse_output(self, text: str):
        """解析LLM的输出，提取Thought和Action。"""
        thought_match = re.search(r"Thought: (.*)", text)
        action_match = re.search(r"Action: (.*)", text)
        thought = thought_match.group(1).strip() if thought_match else None
        action = action_match.group(1).strip() if action_match else None
        return thought, action

    def _parse_action(self, action_text: str):
        """解析Action字符串，提取工具名称和输入。"""
        match = re.match(r"(\w+)\[(.*)\]", action_text)
        if match:
            return match.group(1), match.group(2)
        return None, None
    


In [16]:
# test ReActAgent
import re

if __name__ == "__main__":
    # 初始化 LLM 客户端和工具执行器
    llm_client = HelloAgentsLLM()
    tool_executor = ToolExecutor()
    search_description = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
    tool_executor.registerTool("Search", search_description, search)

    # 创建 ReActAgent 实例
    agent = ReActAgent(llm_client, tool_executor, max_steps=3)

    # 测试问题
    question = "英伟达最新的GPU型号是什么？"
    print("\n===== 测试 ReActAgent =====")
    answer = agent.run(question)
    print("\n最终答案:", answer)


工具 'Search' 已注册。

===== 测试 ReActAgent =====
--- 第 1 步 ---
Thinking with model: gpt-5-nano
Response received: Thought: 我需要通过网络最新信息来确定NVIDIA当前最新的GPU型号，因为这是时效性很强的事实性问题，可能在知识库之外。将进行网页搜索以获取官方公告或权威媒体的最新报道。

Action: Search[NVIDIA latest GPU model 2024 2025 RTX 50 series 5090]
思考: 我需要通过网络最新信息来确定NVIDIA当前最新的GPU型号，因为这是时效性很强的事实性问题，可能在知识库之外。将进行网页搜索以获取官方公告或权威媒体的最新报道。
🎬 行动: Search[NVIDIA latest GPU model 2024 2025 RTX 50 series 5090]
🔍 正在执行 [SerpApi] 网页搜索: NVIDIA latest GPU model 2024 2025 RTX 50 series 5090
👀 观察: [1] GeForce RTX 5090 Graphics Cards
The GeForce RTX 5090 is powered by the NVIDIA Blackwell architecture and equipped with 32 GB of super-fast GDDR7 memory, so you can do it all. Starting at $1999.

[2] GeForce RTX 5090 D v2 显卡| NVIDIA
NVIDIA® GeForce RTX™ 5090 D v2 可为游戏玩家和创作者带来全新玩法。它具有强大AI 算力，能够处理前沿的模型，应对挑战性的创意工作流。在游戏体验中，支持全景光线追踪 ...

[3] GeForce RTX 50 Series Graphics Cards
Equipped with a massive level of AI horsepower, the RTX 50 Series enables new experiences and next-level graphics fi

## Plan-and-Solve

感觉不像一个智能体，更像一个依赖长上下文记忆的普通LLM。这种循环执行的方式，和ReAct的边想边做有本质区别。这种对线性的简单任务支持还好，稍微复杂一点应该是不行的。

如果是复杂的计划和任务，个人觉得维护几个AI可以access的外部项目文档是更稳妥的。例如清晰的需求文档，任务分解文档，项目进度文档等。让AI可以随时查询和更新这些文档，应该会比单纯依赖上下文记忆更靠谱。

这个智能体跳过了。

## Reflection

Reflection 机制比较有意思，感觉更好地“模仿”了人类思考的流程。

核心工作流程可以概括为一个简洁的三步循环：执行 -> 反思 -> 优化。

- 执行 (Execution)：首先，智能体使用我们熟悉的方法（如 ReAct 或 Plan-and-Solve）尝试完成任务，生成一个初步的解决方案或行动轨迹。这可以看作是“初稿”。
- 反思 (Reflection)：接着，智能体进入反思阶段。它会调用一个独立的、或者带有特殊提示词的大语言模型实例，来扮演一个“评审员”的角色。这个“评审员”会审视第一步生成的“初稿”，并从多个维度进行评估，例如：
  - 事实性错误：是否存在与常识或已知事实相悖的内容？
  - 逻辑漏洞：推理过程是否存在不连贯或矛盾之处？
  - 效率问题：是否有更直接、更简洁的路径来完成任务？
  - 遗漏信息：是否忽略了问题的某些关键约束或方面？ 根据评估，它会生成一段结构化的反馈 (Feedback)，指出具体的问题所在和改进建议。
- 优化 (Refinement)：最后，智能体将“初稿”和“反馈”作为新的上下文，再次调用大语言模型，要求它根据反馈内容对初稿进行修正，生成一个更完善的“修订稿”。